<a href="https://colab.research.google.com/github/Hyperyop/mango_classification_challenge/blob/main/mango.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://dphi-live.s3.eu-west-1.amazonaws.com/dataset/mangopests_dataset.zip
!7z x mangopests_dataset.zip -o/content/mangopests
!rm mangopests_dataset.zip
#code for download and extractions of dataset

--2021-12-22 17:24:52--  https://dphi-live.s3.eu-west-1.amazonaws.com/dataset/mangopests_dataset.zip
Resolving dphi-live.s3.eu-west-1.amazonaws.com (dphi-live.s3.eu-west-1.amazonaws.com)... 52.218.57.40
Connecting to dphi-live.s3.eu-west-1.amazonaws.com (dphi-live.s3.eu-west-1.amazonaws.com)|52.218.57.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5305899330 (4.9G) [application/zip]
Saving to: ‘mangopests_dataset.zip’

mangopests_dataset. 100%[===================>]   4.94G  28.3MB/s    in 3m 0s   

2021-12-22 17:27:53 (28.0 MB/s) - ‘mangopests_dataset.zip’ saved [5305899330/5305899330]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 5305899330 bytes (5061 MiB)

Extracting archive: mangopests_dataset.zip
  0% 256 Open   

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import keras,os
import csv
from tensorflow.python.client import device_lib
import numpy as np
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1186863906743971346
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11320098816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6316559293365744057
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]


In [ ]:
import shutil
with open('/content/mangopests/mangopests_dataset/Training_set.csv','r') as file:
  data = csv.reader(file)
  data.__next__()
  for row in data:
    dest_fpath='/content/drive/MyDrive/Colab Notebooks/trainset/'+row[1]+"/"+row[0]
    src_fpath='/content/mangopests/mangopests_dataset/train/'+row[0]
    os.makedirs(os.path.dirname(dest_fpath), exist_ok=True)
    shutil.copyfile(src_fpath, dest_fpath)
    print(row)

In [2]:
trdata  = ImageDataGenerator(rotation_range=15,rescale=1.0/255.0,height_shift_range=0.15,width_shift_range=0.15,fill_mode="constant",vertical_flip=True,validation_split=0.2)
traindata = trdata.flow_from_directory(directory="/content/drive/MyDrive/Colab Notebooks/trainset",target_size=(224,224), subset="training")
valdata = trdata.flow_from_directory(directory="/content/drive/MyDrive/Colab Notebooks/trainset",target_size=(224,224), subset="validation")

Found 26040 images belonging to 16 classes.
Found 6510 images belonging to 16 classes.


In [3]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

In [12]:

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))
with strategy.scope():#using multiple gpus to train
    base_model = VGG19(weights="imagenet", include_top=False, input_shape=(224,224,3)) #TODO change input shape
    #base_model.trainable = False 
    for layer in base_model.layers[:-2]:
            layer.trainable = False
            ## Not trainable weights
    base_model.summary()

    flatten_layer = layers.Flatten()
    dense_layer_1 = layers.Dense(4096, activation='relu')
    dense_layer_2 = layers.Dense(4096, activation='relu')
    dropout_layer=layers.Dropout(0.2)
    prediction_layer = layers.Dense(16, activation='softmax')


    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer_1,
        dense_layer_2,
        prediction_layer
    ])
    model.summary()


    # In[8]:



    opt= tf.keras.optimizers.Adam(learning_rate=0.0001) #TODO change learning rates
    model.compile(
        opt,
        loss='categorical_crossentropy',
        metrics=['accuracy'],
    )


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=4,  restore_best_weights=True)

model.fit(traindata, epochs=20, callbacks=[es],validation_data=valdata)


# In[9]:


loss, accur = model.evaluate(valdata)
print(loss,accur)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
    

In [14]:
if(accur>0.92):
    model.save("/content/drive/MyDrive/Colab Notebooks/mango.model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/mango.model/assets


In [4]:
from PIL import Image

In [5]:
class_names = traindata.class_indices.keys()
class_names=list(class_names)
print(class_names)

['apoderus_javanicus', 'aulacaspis_tubercularis', 'ceroplastes_rubens', 'cisaberoptus_kenyae', 'dappula_tertia', 'dialeuropora_decempuncta', 'erosomyia_sp', 'icerya_seychellarum', 'ischnaspis_longirostris', 'mictis_longicornis', 'neomelicharia_sparsa', 'normal', 'orthaga_euadrusalis', 'procontarinia_matteiana', 'procontarinia_rubus', 'valanga_nigricornis']


In [6]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/mango.model')

In [ ]:
file2=open("results.txt",'w')
file2.write("filename,label\n")
with open('/content/mangopests/mangopests_dataset/Testing_set.csv','r') as file:
  data = csv.reader(file)
  data.__next__()
  for row in data:
    A=Image.open('/content/mangopests/mangopests_dataset/test/'+row[0])
    A=A.resize((224,224))
    A=np.array(A,dtype=np.float32)
    A/=255.0
    a=model.predict(np.array([A]))
    file2.write(row[0]+','+class_names[np.argmax(a)]+'\n')